In [10]:
using DrWatson
@quickactivate "DifferentiableEvaporation"
using Symbolics, Nemo

The goal of the current notebook is to extend upon the derivations of a series network from:

- "Evaporation from sparse crops-an energy combination theory" by [Shuttleworth and Wallace (1985)](https://doi.org/10.1002/qj.49711146910) (S&W85)
- "Evaporation from Heterogeneous and Sparse Canopies: On the Formulations Related to Multi-Source Representations" by [Lhomme et al. (2012)](https://doi.org/10.1007/s10546-012-9713-x) (Lh12)

Start by defining the different variables


In [11]:
@variables begin
    As # Available energy at the surface (soil) (W/m²)
    Ac # Available energy at the canopy (W/m²)
    ρ # Density of air (kg/m³)
    cp # Specific heat of air (J/kg/K)
    Vpd_a # Vapor pressure deficit (Pa) at observation height
    γ # Psychrometric constant (Pa/K)
    Δ # Slope of the saturation vapor pressure curve (Pa/K) 
    r_aa # Aerodynamic resistance (s/m) between source height and observation height
    r_ac # Aerodynamic resistance (s/m) between canopy and source height
    r_as # Aerodynamic resistance (s/m) between soil and source height
    r_ss # Surface resistance (s/m) of soil
    r_sc # Surface resistance (s/m) of canopy
    λE # Latent heat flux from the complete canopy (W/m²)
    Ra
    Rs
    Rc
    f_wet # Fraction of canopy that is wet (-)
end

17-element Vector{Num}:
    As
    Ac
     ρ
    cp
 Vpd_a
     γ
     Δ
  r_aa
  r_ac
  r_as
  r_ss
  r_sc
    λE
    Ra
    Rs
    Rc
 f_wet

## Reference case from Lhomme et al. (2012)

Let's symbolically solve the equations governing the standard case of a two source model, as depicted in Figure 1 of Lh12.


In [12]:
A = As + Ac # Total available energy
Vpd_0 = Vpd_a + (Δ * A - (Δ + γ) * λE) * r_aa / (ρ * cp) # Vapor pressure deficit at source height, (7) of Lh12
λE_s = (Δ * As + ρ * cp * Vpd_0 / r_as) / (Δ + γ * (1 + r_ss / r_as)) # Latent heat flux from the soil
λE_c = (Δ * Ac + ρ * cp * Vpd_0 / r_ac) / (Δ + γ * (1 + r_sc / r_ac)) # Latent heat flux from the canopy
equation = λE - (λE_s + λE_c) ~ 0 # Latent heat flux balance, (1) of Lh12
display(equation)
sol = symbolic_solve(equation, λE)
sol[1];

((-(Vpd_a + (((Ac + As)*Δ - (Δ + γ)*λE)*r_aa) / (cp*ρ))*cp*ρ) / r_ac - Ac*Δ) / (Δ + (1 + r_sc / r_ac)*γ) + ((-(Vpd_a + (((Ac + As)*Δ - (Δ + γ)*λE)*r_aa) / (cp*ρ))*cp*ρ) / r_as - As*Δ) / (Δ + (1 + r_ss / r_as)*γ) + λE ~ 0

Check if the solution matches with the solution as given by equation 16 of Lh12. Note that f (folliage) replaced by c (canopy). First this reference solution is implemented below.

In [13]:
Rc = r_sc + (1 + Δ / γ) * r_ac # (20) of Lh12
Rs = r_ss + (1 + Δ / γ) * r_as # (21) of Lh12
Ra = (1 + Δ / γ) * r_aa # (22) of Lh12
Ps = (r_aa * Rc) / (Rc * Rs + Ra * Rc + Rs * Ra) # (18) of Lh12
Pc = (r_aa * Rs) / (Rc * Rs + Ra * Rc + Rs * Ra) # (19) of Lh12
λEp = (Δ * A + ρ * cp * Vpd_a / r_aa) / (Δ + γ) # (17) of Lh12
λE_lhomme = ((Δ + γ) / γ) * (Pc + Ps) * λEp +
            (Δ / (γ * r_aa)) * (Pc * Ac * r_ac + Ps * As * r_as); # (16) of Lh12

Now symbolically check the equality between the two solutions by subtracting the two equations from each other after simplifying them. 

In [14]:
λE_computer_simple = simplify(sol[1])
λE_lhomme_simple = simplify(λE_lhomme)
print(isequal(simplify(λE_computer_simple - λE_lhomme_simple), 0))

true

## Own derivation: adding interception 

Idea is to extend the fluxes so that from a leave, there are now 2 fluxes in parallel:
1. Interception flux from wet fraction of the leaf ($f_{wet}$) with a zero surface resistance
2. Evaporation flux from the dry fraction of the leaf ($1 - f_{wet}$)

In [15]:
λE_t = (1 - f_wet) * (Δ * Ac + ρ * cp * Vpd_0 / r_ac) / (Δ + γ * (1 + r_sc / r_ac))
λE_i = f_wet * (Δ * Ac + ρ * cp * Vpd_0 / r_ac) / (Δ + γ)
equation_interception = λE - (λE_t + λE_i + λE_s) ~ 0
display(equation_interception)
sol_interception = symbolic_solve(equation_interception, λE);

(-(((Vpd_a + (((Ac + As)*Δ - (Δ + γ)*λE)*r_aa) / (cp*ρ))*cp*ρ) / r_ac + Ac*Δ)*f_wet) / (Δ + γ) + ((-(Vpd_a + (((Ac + As)*Δ - (Δ + γ)*λE)*r_aa) / (cp*ρ))*cp*ρ) / r_as - As*Δ) / (Δ + (1 + r_ss / r_as)*γ) + (-(((Vpd_a + (((Ac + As)*Δ - (Δ + γ)*λE)*r_aa) / (cp*ρ))*cp*ρ) / r_ac + Ac*Δ)*(1 - f_wet)) / (Δ + (1 + r_sc / r_ac)*γ) + λE ~ 0

Below the solution from the manual derivation is given.

In [16]:
Ri = (1 + Δ / γ) * r_ac # In analogy to (20) - (22) of Lh12
X = Rc * Ri + (1 - f_wet) * Rs * Ri + f_wet * Rs * Rc
DE = Rs * Rc * Ri + Ra * X
λE_check = (X * r_aa / DE) * ((Δ + γ) / γ) * λEp +
           (Δ / γ) * ((Rc * Ri * As * r_as + r_ac * Ac * ((1 - f_wet) * Rs * Ri + f_wet * Rc * Rs)) / DE);

As above, the solution obtained by the computer algebra system will be compared with the solution obtained by hand. 

In [17]:
λE_check_simple = simplify(λE_check)
λE_interception_simple = simplify(sol_interception[1])
print(isequal(simplify(λE_check_simple - λE_interception_simple), 0))

true

There is also a different approach to solve the problem by hand, where the $R_c$ value as defined in (20) of Lh12 is adapted for the current case (denoted as $R_c^*$). Once this is defined, we can use the same equations as in the reference case (Section 2.2 of Lh12).

In [18]:
Rcstar = ((1 - f_wet) / Rc + f_wet / Ri)^-1
Psstar = simplify((r_aa * Rcstar) / (Rcstar * Rs + Ra * Rcstar + Rs * Ra)) # Analogous to (18) of Lh12
Pcstar = simplify((r_aa * Rs) / (Rcstar * Rs + Ra * Rcstar + Rs * Ra)) # Analogous to (19) of Lh12
λE_check_alt = simplify(((Δ + γ) / γ) * (Pcstar + Psstar) * λEp) +
               simplify((Δ / (γ * r_aa)) * (Pcstar * Ac * r_ac + Psstar * As * r_as)) # Analogous to (16) of Lh12
λE_check_alt_simple = simplify(λE_check_alt)
print(isequal(simplify(λE_interception_simple - λE_check_alt_simple), 0))

true